In [1]:
import os
import time
import pymysql
import requests
from multiprocessing import cpu_count
from multiprocessing.pool import ThreadPool

"""
创建文件夹
"""     
def mkd(url):
    if os.path.exists(url) == False:
        os.makedirs(url)

"""
创建根文件夹
"""
cur_time = time.strftime("%Y%m%d%H%M%S", time.gmtime())   
path_root = os.path.join("/opt/filebrowser/data/dataset", cur_time)
mkd(path_root)
          
"""
图片数据集
"""
image_url, fns_url = [], []
def generate_image_set(image, file_path):
    for item in image:
        image_url.append(item["uri"])
        fns_url.append(os.path.join(file_path, item["file_name"]))

"""
参数start_time,end_time设置按时间段导出,时间格式为%Y-%m-%d
"""   
start_time = "2020-12-25"
end_time = ""
base_sql = "select concat('https://ty.cutcckms.com/zhjl', b.RELATIVE_PATH, b.file_name) uri, b.file_name, b.create_date from zhjl_itemtaskinspect a \
            left join base_file_upload_info b on a.fileid = b.file_id \
            where a.del_flag = 0 and b.del_flag = 0" 
file_sql = ""
if len(start_time) == 0 or len(end_time) == 0:
    file_sql = base_sql
else:
    file_sql = base_sql + " and b.create_date>= '" + start_time + "' and b.create_date<='" + end_time + "'"

connection = pymysql.connect(
        host="rm-bp1buvmr77jdglr16lo.mysql.rds.aliyuncs.com",
        user="",
        passwd="",
        database="",
        port=3306,
        charset="utf8",
        cursorclass=pymysql.cursors.DictCursor)
cursor = connection.cursor()

# 任务模板
sql = "select id, code, name from zhjl_tasktemplate where del_flag = 0 and property = 2"
cursor.execute(sql)
template = cursor.fetchall()

for template_item in template:
    # 任务
    task_sql = "select id, code, name from zhjl_task where del_flag = 0 and templateid = " + str(template_item["id"])
    cursor.execute(task_sql)
    task = cursor.fetchall()
    template_item["task"] = task
    for task_item in task:
        # 检查项
        inspect_sql = "select id, code, name from zhjl_taskinspect where del_flag = 0 and taskid = " + str(task_item["id"])
        cursor.execute(inspect_sql)
        inspect = cursor.fetchall()
        task_item["inspect"] = inspect
        for inspect_item in inspect:
            # 图片文件存放路径
            path_file = os.path.join(str(template_item["id"]) +"_"+ template_item["name"], 
                                 str(task_item["id"]) + "_" + task_item["name"],
                                 str(inspect_item["id"]) + "_" + inspect_item["name"])
#             path_file = os.path.join(str(template_item["id"]) +"_"+ template_item["name"], 
#                      str(task_item["id"]) + "_" + task_item["name"],
#                      str(inspect_item["id"]) + "_" )
            temp_file = os.path.join(path_root, path_file)
            # 创建文件存放路径
            mkd(temp_file)
            # 图片
            image_sql = file_sql + " and a.inspectid = " + str(inspect_item["id"])
            cursor.execute(image_sql)
            image = cursor.fetchall()
            inspect_item["image"] = image
            generate_image_set(image, temp_file)            
connection.close()

In [2]:
inputs = zip(image_url, fns_url)

def download_url(args):
    t0 = time.time()
    url, fn = args[0], args[1]
    try:
        r = requests.get(url)
        with open(fn, 'wb') as f:
            f.write(r.content)
        return(url, time.time() - t0)
    except Exception as e:
        print('Exception in download_url():', e)
        
def download_parallel(args):
#     cpus = cpu_count()
    results = ThreadPool(300).imap_unordered(download_url, args)
#     for result in results:
#         print('url:', result[0], 'time (s):', result[1])
        
download_parallel(inputs)